<a href="https://colab.research.google.com/github/Storm00212/QUANT/blob/main/MILLENIUM_ALGO_N%26Q.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Colab code to simulate a quant finance model with a risk/reward surface and performance charts.
# This is a simplified simulation based on the provided image description.
# Assumptions:
# - The 3D surface represents a risk/reward landscape (e.g., expected return vs. risk vs. time or parameters).
# - The model applies a weighting function during specific trading hours (9:30-12:30 ET).
# - For simplicity, we use synthetic data to mimic S&P 500 returns and an "algo" that outperforms it.
# - The equation is displayed as text in the plot.
# - Run this in Google Colab with %matplotlib inline.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import datetime

In [2]:
# Enable inline plotting
%matplotlib inline

In [3]:
# Function to generate a synthetic risk/reward surface
def generate_surface():
    x = np.linspace(0, 200, 100)
    y = np.linspace(0, 50, 100)
    X, Y = np.meshgrid(x, y)
    # Synthetic function to mimic the wavy surface in the image (adjust as needed)
    Z = 20 * np.sin(X / 20) * np.cos(Y / 10) + 30 + np.random.normal(0, 5, X.shape)
    return X, Y, Z

# Function to simulate performance data
def simulate_performance(num_days=365):
    np.random.seed(42)  # For reproducibility
    dates = [datetime.date(2025, 1, 1) + datetime.timedelta(days=i) for i in range(num_days)]

    # Synthetic S&P 500 buy-and-hold: moderate returns with volatility
    sp_returns = np.random.normal(0.0005, 0.01, num_days)  # Daily returns ~0.05% mean, 1% std
    sp_cumulative = 15000 * np.cumprod(1 + sp_returns)

    # Synthetic Millennium Algo: higher returns, but only "trades" in morning hours (simulated by boosting returns)
    # Assume the weighting u(t) amplifies returns during "trading window" (simulated as random boost)
    algo_returns = sp_returns + np.random.normal(0.0008, 0.005, num_days)  # Higher mean
    algo_cumulative = 15000 * np.cumprod(1 + algo_returns)

    return dates, sp_cumulative, algo_cumulative

In [4]:
# Generate data
X, Y, Z = generate_surface()
dates, sp_values, algo_values = simulate_performance()